In [1]:
import pandas
import cvxopt
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import check_random_state
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

In [3]:
def print_metrics(Method, X, y, folds = 5, average = 'macro'):
  kf = StratifiedKFold(n_splits = folds, random_state = 123, shuffle = True)
  precision = np.zeros(folds)   
  recall = np.zeros(folds)  
  testAc = np.zeros(folds)
  trainAc = np.zeros(folds)
  for i, (trainI, valI) in enumerate(kf.split(X, y)):
    XT, yT = X.loc[trainI].to_numpy(), y.loc[trainI].to_numpy()
    XV, yV = X.loc[valI].to_numpy(), y.loc[valI].to_numpy()
    Method.fit(XT, yT)
    yP = Method.predict(XV)
    yTP = Method.predict(XT)
    precision[i] = precision_score(yV, yP, average = average)
    recall[i] = recall_score(yV, yP, average = average)
    trainAc[i] = accuracy_score(yT, yTP)
    testAc[i] = accuracy_score(yV, yP)    
  print("precision:", precision.mean(), "\nrecall:", recall.mean(), "\n\ntrain_accuracy:", trainAc.mean(), "\ntest_accuracy:", testAc.mean())

In [4]:
class LogReg():
  def __init__(self, accuracy = 0.01, iters = 1000):
    self.it = iters
    self.ac = accuracy
        
  def Sigmoid(self, x):
    return 1 / (1 + np.exp(-x))

  def AddIntercept(self, X):
    return np.concatenate((np.ones((X.shape[0], 1)), X), axis = 1)

  def Loss(self, h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

  def fit(self, X, y):
    X = self.AddIntercept(X)
    self.wei = np.zeros(X.shape[1])
    for _ in range(self.it):
      h = self.Sigmoid(np.dot(X, self.wei))
      grad = np.dot(X.T, (h - y)) / y.size
      self.wei -= self.ac * grad
    pass

  def predict(self, X):
    X = self.AddIntercept(X)
    return self.Sigmoid(np.dot(X, self.wei))

In [5]:
class KNN():
  def __init__(self, neighbors = 5):
    self.nei = neighbors
    
  def fit(self, X, y):
    self.X = X
    self.y = y.reshape((y.shape[0], 1))

  def Distances(self, p):
    t = self.X - p
    return np.sqrt((t ** 2).sum(1))

  def predict(self, X):
    n = X.shape[0]
    pred = np.zeros((n, 1))
    for i in range(n):
      d = self.Distances(X[i])
      sorted = self.y[np.argsort(d)].flatten()
      if sorted[:self.nei].sum() > self.nei / 2:
        pred[i] = 1.0
    return pred

In [6]:
class Node():
  def __init__(self, predClass):
    self.predClass = predClass
    self.index = 0
    self.threshold = 0
    self.left = None
    self.right = None

class DecisionTree():
  def __init__(self, maxDepth = 1, rf = False):
    self.maxDepth = maxDepth
    self.rf = rf

  def fit(self, X, y, maxFeatures = None):
    self.classes = len(set(y))
    if not self.rf:
      Features = X.shape[1]
    else:
      ind = np.random.choice(X.shape[0], X.shape[0])
      X, y = X[tuple([ind])], y[tuple([ind])]
      if maxFeatures is None:
        Features = np.sqrt(X.shape[1]).astype(int)
      else:
        Features = maxFeatures
    self.features = np.sort(np.random.choice(X.shape[1], Features, replace = False))
    self.tree = self.GrowTree(X, y)

  def predict(self, X):
    list = []
    for inputs in X:
      node = self.tree
      while node.left:
        if inputs[node.featureIndex] < node.threshold:
          node = node.left
        else:
          node = node.right
      list.append(node.predClass)
    return list

  def Split(self, X, y):
    m = y.size
    if m <= 1:
      return None, None
    parent = [np.sum(y == c) for c in range(self.classes)]
    bestGini = 1.0 - sum((n / m) ** 2 for n in parent)
    bestIdx, bestThr = None, None
    for idx in self.features:
      thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
      left = [0] * self.classes
      right = parent.copy()
      for i in range(1, m):
        c = classes[i - 1]
        left[c] += 1
        right[c] -= 1
        giniLeft = 1.0 - sum((left[x] / i) ** 2 for x in range(self.classes))
        giniRight = 1.0 - sum((right[x] / (m - i)) ** 2 for x in range(self.classes))
        gini = (i * giniLeft + (m - i) * giniRight) / m
        if thresholds[i] == thresholds[i - 1]:
          continue
        if gini < bestGini:
          bestGini = gini
          bestIdx = idx
          bestThr = (thresholds[i] + thresholds[i - 1]) / 2
    return bestIdx, bestThr

  def GrowTree(self, X, y, depth = 0):
    samplesPerClass = [np.sum(y == i) for i in range(self.classes)]
    predClass = np.argmax(samplesPerClass)
    node = Node(predClass = predClass)
    if depth < self.maxDepth:
      idx, thr = self.Split(X, y)
      if idx is not None:
        indicesLeft = X[:, idx] < thr
        XLeft, yLeft = X[indicesLeft], y[indicesLeft]
        XRight, yRight = X[~indicesLeft], y[~indicesLeft]
        node.featureIndex = idx
        node.threshold = thr
        node.left = self.GrowTree(XLeft, yLeft, depth + 1)
        node.right = self.GrowTree(XRight, yRight, depth + 1)
    return node



In [7]:
class SVM:
  def __init__(self, C = 1, maxIter = 100, eps = 0.01, randomState = None, verbose = 0):
    self.C = C
    self.maxIter = maxIter
    self.eps = eps
    self.randomState = randomState
    self.verbose = verbose

  def PartialGradient(self, X, y, i):
    g = np.dot(X[i], self.coef.T) + 1
    return g

  def Violation(self, g, y, i):
    smallest = np.inf
    for k in range(g.shape[0]):
      if k == y[i] and self.dualCoef[k, i] >= self.C:
        continue
      elif k != y[i] and self.dualCoef[k, i] >= 0:
        continue
      smallest = min(smallest, g[k])
    return g.max() - smallest

  def Solver(self, g, y, norms, i):
    Ci = np.zeros(g.shape[0])
    Ci[y[i]] = self.C
    beta_hat = norms[i] * (Ci - self.dualCoef[:, i]) + g / norms[i]
    z = self.C * norms[i]
    beta = projection_simplex(beta_hat, z)
    return Ci - self.dualCoef[:, i] - beta / norms[i]

  def fit(self, X, y):
    samples, features = X.shape
    classes = 4
    self.dualCoef = np.zeros((classes, samples), dtype = np.float64)
    self.coef = np.zeros((classes, features))
    norms = np.sqrt(np.sum(X ** 2, axis = 1))
    rs = check_random_state(self.randomState)
    ind = np.arange(samples)
    rs.shuffle(ind)
    violationInit = None
    for it in range(self.maxIter):
      violationSum = 0
      for idx in range(samples):
        i = ind[idx]
        if norms[i] == 0:
          continue
        g = self.PartialGradient(X, y, i)
        v = self.Violation(g, y, i)
        violationSum += v
        if v < 1e-12:
          continue
        delta = self.Solver(g, y, norms, i)
        self.coef += (delta * X[i][:, np.newaxis]).T
        self.dualCoef[:, i] += delta
      if it == 0:
        violationInit = violationSum
      vratio = violationSum / violationInit
      if vratio < self.eps:
        break
    return self

  def predict(self, X):
    decision = np.dot(X, self.coef.T)
    pred = decision.argmax(axis = 1)
    return pred

# **For dataset 1**

In [8]:
anime = pandas.read_csv("data_new.csv")
anime["episode_class"].unique()

array([4, 3, 1, 2, 0])

In [9]:
needed = ["episodes"]
y = anime["episode_class"].map({0 : 0, 1 : 1, 2 : 2, 3 : 3, 4 : 4})
X = pandas.get_dummies(anime[needed])

In [10]:
%%time
print_metrics(LogReg(), X, y)

precision: 0.13337062937062938 
recall: 0.2 

train_accuracy: 0.6668530002662953 
test_accuracy: 0.6668531468531469
CPU times: user 443 ms, sys: 398 µs, total: 444 ms
Wall time: 449 ms


In [11]:
%%time
print_metrics(LogisticRegression(), X, y)

precision: 0.9988360974194164 
recall: 0.9906994535519125 

train_accuracy: 0.9960871652664796 
test_accuracy: 0.9960866507793881
CPU times: user 1.57 s, sys: 1.18 s, total: 2.75 s
Wall time: 1.4 s


In [12]:
%%time
print_metrics(KNN(neighbors = 4), X, y)

precision: 0.336420846106683 
recall: 0.4 

train_accuracy: 0.6892118606193431 
test_accuracy: 0.6892120170332461
CPU times: user 3.41 s, sys: 2.06 ms, total: 3.41 s
Wall time: 3.41 s


In [13]:
%%time
print_metrics(KNeighborsClassifier(n_neighbors = 4), X, y)

precision: 1.0 
recall: 1.0 

train_accuracy: 1.0 
test_accuracy: 1.0
CPU times: user 519 ms, sys: 0 ns, total: 519 ms
Wall time: 521 ms


In [14]:
%%time
print_metrics(DecisionTree(maxDepth = 5), X, y)

precision: 1.0 
recall: 1.0 

train_accuracy: 1.0 
test_accuracy: 1.0
CPU times: user 523 ms, sys: 2.45 ms, total: 526 ms
Wall time: 528 ms


In [15]:
%%time
print_metrics(DecisionTreeClassifier(max_depth = 5), X, y)

precision: 1.0 
recall: 1.0 

train_accuracy: 1.0 
test_accuracy: 1.0
CPU times: user 38.7 ms, sys: 726 µs, total: 39.4 ms
Wall time: 44.2 ms


In [16]:
%%time
print_metrics(SVM(), X, y)

precision: 0.004471774036019846 
recall: 0.2 

train_accuracy: 0.022358860353047738 
test_accuracy: 0.02235887018009923
CPU times: user 22.9 s, sys: 10.1 ms, total: 22.9 s
Wall time: 22.9 s


In [17]:
%%time
print_metrics(SVC(decision_function_shape = 'ovr'), X, y)

precision: 0.8232503238603044 
recall: 0.7514443064525033 

train_accuracy: 0.9576577031759943 
test_accuracy: 0.9566765636598038
CPU times: user 508 ms, sys: 1.93 ms, total: 510 ms
Wall time: 511 ms


# **For dataset 2**

In [18]:
google = pandas.read_csv("google_new.csv")
google["Rating"].unique()

array([4.1, 3.9, 4.7, 4.5, 4.3, 4.4, 3.8, 4.2, 4.6, 3.2, 4. , 4.8, 4.9,
       3.6, 3.7, 3.3, 3.4, 3.5, 3.1, 5. , 2.6, 3. , 1.9, 2.5, 2.8, 2.7,
       1. , 2.9, 2.3, 2.2, 1.7, 2. , 1.8, 2.4, 1.6, 2.1, 1.4, 1.5, 1.2])

In [19]:
needed = ["Reviews", "Installs"]
yy = google["Rating"].map({4.1 : 0, 3.9 : 1, 4.7 : 2, 4.5 : 3, 4.3 : 4, 4.4 : 5, 3.8 : 6, 4.2 : 7, 4.6 : 8, 3.2 : 9, 4. : 10, 4.8 : 11, 4.9 : 12,
                 3.6 : 13, 3.7 : 14, 3.3 : 15, 3.4 : 16, 3.5 : 17, 3.1 : 18, 5. : 19, 2.6 : 20, 3. : 21, 1.9 : 22, 2.5 : 23, 2.8 : 24, 2.7 : 25,
                 1. : 26, 2.9 : 27, 2.3 : 28, 2.2 : 29, 1.7 : 30, 2. : 31, 1.8 : 32, 2.4 : 33, 1.6 : 34, 2.1 : 35, 1.4 : 36, 1.5 : 37, 1.2 : 38})
XX = pandas.get_dummies(google[needed])

In [20]:
%%time
print_metrics(LogReg(), XX, yy)

precision: 0.001108342592481498 
recall: 0.026892682155840054 

train_accuracy: 0.041212898545137946 
test_accuracy: 0.04121291098979431
CPU times: user 2.09 s, sys: 1.51 s, total: 3.6 s
Wall time: 1.83 s


In [21]:
%%time
print_metrics(LogisticRegression(), XX, yy)

precision: 0.016910236352581272 
recall: 0.03152216371762873 

train_accuracy: 0.12932421924459983 
test_accuracy: 0.12865696372822577
CPU times: user 20.8 s, sys: 14.9 s, total: 35.6 s
Wall time: 18.1 s


In [22]:
%%time
print_metrics(KNN(neighbors = 6), XX, yy)

precision: 0.011634889424356084 
recall: 0.026967074009012087 

train_accuracy: 0.04190689740750381 
test_accuracy: 0.04142641514164379
CPU times: user 35.7 s, sys: 9.54 ms, total: 35.7 s
Wall time: 35.7 s


In [23]:
%%time
print_metrics(KNeighborsClassifier(n_neighbors = 6), XX, yy)

precision: 0.07654738391427982 
recall: 0.07233332206055851 

train_accuracy: 0.3772420658996983 
test_accuracy: 0.18129203345183278
CPU times: user 1.55 s, sys: 1.17 ms, total: 1.55 s
Wall time: 1.56 s


In [24]:
%%time
print_metrics(DecisionTree(maxDepth = 7), XX, yy)

precision: 0.05606844547196253 
recall: 0.05876589853021934 

train_accuracy: 0.1969356948849002 
test_accuracy: 0.16250155982817105
CPU times: user 21.4 s, sys: 11.9 ms, total: 21.4 s
Wall time: 21.4 s


In [25]:
%%time
print_metrics(DecisionTreeClassifier(max_depth = 7), XX, yy)

precision: 0.056105804240517254 
recall: 0.05876589853021934 

train_accuracy: 0.1969356948849002 
test_accuracy: 0.16250155982817105
CPU times: user 110 ms, sys: 956 µs, total: 111 ms
Wall time: 112 ms


In [26]:
%%time
print_metrics(SVM(), XX, yy)

precision: 0.00203293833082738 
recall: 0.026892682155840054 

train_accuracy: 0.07559256765649007 
test_accuracy: 0.07559254951991481
CPU times: user 1min, sys: 46.4 ms, total: 1min
Wall time: 1min


In [27]:
%%time
print_metrics(SVC(decision_function_shape = 'ovr'), XX, yy)

precision: 0.016579660847247178 
recall: 0.027550303880359388 

train_accuracy: 0.1229446860850159 
test_accuracy: 0.11968802296978746
CPU times: user 42 s, sys: 101 ms, total: 42.1 s
Wall time: 42.1 s
